In [0]:
import pandas as pd

from google.colab import drive

drive.mount('/content/gdrive/')
pollution_data = pd.read_csv('/content/gdrive/My Drive/AI_NoteBook/PROJECT/Import_File/PlasticMarinePollutionDataset.csv')

pollution_data.rename(columns = {'CD1_(/km^2)' : 'Count_Density_Class1Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD2_(/km^2)' : 'Count_Density_Class2Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD3_(/km^2)' : 'Count_Density_Class3Plastic'}, inplace = True)
pollution_data.rename(columns = {'CD4 _(/km^2)' : 'Count_Density_Class4Plastic'}, inplace = True)

pollution_data.rename(columns = {'WD1_(g/km^2)' : 'Weight_Density_Class1Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD2_(g/km^2)' : 'Weight_Density_Class2Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD3_(g/km^2)' : 'Weight_Density_Class3Plastic'}, inplace = True)
pollution_data.rename(columns = {'WD4_(g/km^2)' : 'Weight_Density_Class4Plastic'}, inplace = True)

pollution_data['Count_Density_Class1Plastic'] = pollution_data.Count_Density_Class1Plastic.str.replace(' ', '')
pollution_data['Count_Density_Class2Plastic'] = pollution_data.Count_Density_Class2Plastic.str.replace(' ', '')
pollution_data['Count_Density_Class3Plastic'] = pollution_data.Count_Density_Class3Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class2Plastic'] = pollution_data.Weight_Density_Class2Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class3Plastic'] = pollution_data.Weight_Density_Class3Plastic.str.replace(' ', '')
pollution_data['Weight_Density_Class4Plastic'] = pollution_data.Weight_Density_Class4Plastic.str.replace(' ', '')

pollution_data['Count_Density_Class1Plastic'] = pollution_data.Count_Density_Class1Plastic.str.replace(',', '').astype(float)
pollution_data['Count_Density_Class2Plastic'] = pollution_data.Count_Density_Class2Plastic.str.replace(',', '').astype(float)
pollution_data['Count_Density_Class3Plastic'] = pollution_data.Count_Density_Class3Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class2Plastic'] = pollution_data.Weight_Density_Class2Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class3Plastic'] = pollution_data.Weight_Density_Class3Plastic.str.replace(',', '').astype(float)
pollution_data['Weight_Density_Class4Plastic'] = pollution_data.Weight_Density_Class4Plastic.str.replace(',', '').astype(float)

pollution_data = pollution_data.drop_duplicates(keep = 'first')

pollution_data['Data'] = pollution_data.Date.astype(str)

pollution_data_4 = pollution_data[['Date','Count_Density_Class4Plastic']]

pollution_data_4 = pollution_data_4.dropna(how = 'any')

pollution_data_4['Date'] = pd.to_datetime(pollution_data_4.Date)

pollution_data_4_GroupByDate = pollution_data_4.groupby('Date').Count_Density_Class4Plastic.sum()

pollution_data_4_GroupByDate = pd.DataFrame(pollution_data_4_GroupByDate)

pollution_data_4_GroupByDate ['Date'] = pollution_data_4_GroupByDate.index

pollution_data_4_GroupByDate.info()
pollution_data_4_GroupByDate.head(50)

In [0]:
startdate = pd.to_datetime('2008/02/06')

pollution_data_4_GroupByDate ['Days after 2008-02-06'] = (pollution_data_4_GroupByDate.Date).subtract(startdate)

pollution_data_4_GroupByDate ['Days after 2008-02-06']  = pollution_data_4_GroupByDate['Days after 2008-02-06'].astype(str)

pollution_data_4_GroupByDate ['Days after 2008-02-06'] = pollution_data_4_GroupByDate ['Days after 2008-02-06'].str.replace(' days 00:00:00.000000000', '')

pollution_data_4_GroupByDate.info()
pollution_data_4_GroupByDate.head(50)

In [0]:
X = pollution_data_4_GroupByDate ['Days after 2008-02-06'].astype(float)

y = pollution_data_4_GroupByDate ['Count_Density_Class4Plastic']

import matplotlib.pyplot as plt

plt.figure(figsize= (50, 30))

plt.title('Count Density', color = 'w', fontsize = '30')

plt.scatter(X, y, color='gray')
plt.plot(X, y, color='skyblue', linewidth = '2')

plt.xticks(color = 'w', fontsize = '15', rotation = 20)
plt.yticks(color = 'w', fontsize = '30')

plt.show()

In [0]:
import pickle

import numpy as np
from numpy import array

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm

########## [SPLITING INTO TRAINING AND TESTING] ##########
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.2)

########## [CONVERTING SERIES INTO ARRAY] ##########
X_train = X_train.to_numpy()

X_train = X_train.reshape(-1,1)

y_train = y_train.to_numpy()

X_test = X_test.sort_values('index')
X_test = X_test.to_numpy()
X_test = X_test.reshape(-1,1)

y_test = y_test.sort_values('index')
y_test = y_test.to_numpy()

########## [ DEFINE MODEL AND TRAIN  < Degree = 1 > ] ##########
polynomial = PolynomialFeatures(degree=1)

X_train_transformed_for_poly = polynomial.fit_transform(X_train)

poly_linear_model = linear_model.LinearRegression()

poly_linear_model.fit(X_train_transformed_for_poly, y_train)

########## [ PREDICT < Degree = 1 > ] ##########
X_test_transformed_for_poly = polynomial.fit_transform(X_test)

poly_y_predict = poly_linear_model.predict(X_test_transformed_for_poly)

print("Linear Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, poly_y_predict), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, poly_y_predict), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, poly_y_predict), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, poly_y_predict), 2))
print("R2 score =", round(sm.r2_score(y_test, poly_y_predict), 2))

plt.figure(figsize= (50, 30))

plt.title('Count Density', color = 'w', fontsize = '30')

plt.scatter(X, y, color='gray')
plt.plot(X_test, poly_y_predict, color='darkblue', linewidth = '5')

plt.xticks(color = 'w', fontsize = '15', rotation = 20)
plt.yticks(color = 'w', fontsize = '30')

plt.show()

In [0]:
########## [ DEFINE MODEL AND TRAIN  < Degree = 2 > ] ##########
polynomial = PolynomialFeatures(degree=2)

X_train_transformed_for_poly = polynomial.fit_transform(X_train)

poly_linear_model = linear_model.LinearRegression()

poly_linear_model.fit(X_train_transformed_for_poly, y_train)

########## [ PREDICT < Degree = 2 > ] ##########
X_test_transformed_for_poly = polynomial.fit_transform(X_test)

poly_y_predict = poly_linear_model.predict(X_test_transformed_for_poly)

print("Linear Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, poly_y_predict), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, poly_y_predict), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, poly_y_predict), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, poly_y_predict), 2))
print("R2 score =", round(sm.r2_score(y_test, poly_y_predict), 2))

plt.figure(figsize= (50, 30))

plt.title('Count Density', color = 'w', fontsize = '30')

plt.scatter(X, y, color='gray')
plt.plot(X_test, poly_y_predict, color='darkblue', linewidth = '5')

plt.xticks(color = 'w', fontsize = '15', rotation = 20)
plt.yticks(color = 'w', fontsize = '30')

plt.show()

In [0]:
########## [ DEFINE MODEL AND TRAIN  < Degree = 3 > ] ##########
polynomial = PolynomialFeatures(degree=3)

X_train_transformed_for_poly = polynomial.fit_transform(X_train)

poly_linear_model = linear_model.LinearRegression()

poly_linear_model.fit(X_train_transformed_for_poly, y_train)

########## [ PREDICT < Degree = 3 > ] ##########
X_test_transformed_for_poly = polynomial.fit_transform(X_test)

poly_y_predict = poly_linear_model.predict(X_test_transformed_for_poly)

X_test = pd.DataFrame(X_test)
poly_y_predict = pd.DataFrame(poly_y_predict)

print("Linear Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, poly_y_predict), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, poly_y_predict), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, poly_y_predict), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, poly_y_predict), 2))
print("R2 score =", round(sm.r2_score(y_test, poly_y_predict), 2))

plt.figure(figsize= (50, 30))

plt.title('Count Density', color = 'w', fontsize = '30')

plt.scatter(X, y, color='gray')
plt.plot(X_test, poly_y_predict, color='darkblue', linewidth = '5')

plt.xticks(color = 'w', fontsize = '15', rotation = 20)
plt.yticks(color = 'w', fontsize = '30')

plt.show()

In [0]:
########## [ DEFINE MODEL AND TRAIN  < Degree = 10 > ] ##########
polynomial = PolynomialFeatures(degree=10)

X_train_transformed_for_poly = polynomial.fit_transform(X_train)

poly_linear_model = linear_model.LinearRegression()

poly_linear_model.fit(X_train_transformed_for_poly, y_train)

########## [ PREDICT < Degree = 10 > ] ##########
X_test_transformed_for_poly = polynomial.fit_transform(X_test)

poly_y_predict = poly_linear_model.predict(X_test_transformed_for_poly)

print("Linear Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, poly_y_predict), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, poly_y_predict), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, poly_y_predict), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, poly_y_predict), 2))
print("R2 score =", round(sm.r2_score(y_test, poly_y_predict), 2))

plt.figure(figsize= (50, 30))

plt.title('Count Density', color = 'w', fontsize = '30')

plt.scatter(X, y, color='gray')
plt.plot(X_test, poly_y_predict, color='darkblue', linewidth = '5')

plt.xticks(color = 'w', fontsize = '15', rotation = 20)
plt.yticks(color = 'w', fontsize = '30')

plt.show()